In [1]:
import geemap
import ee
import geopandas as gpd

In [2]:
Map = geemap.Map(center=(44.79, 77.92), zoom=7)
Map.add_basemap('Esri.WorldImagery')
Map

Map(center=[44.79, 77.92], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [3]:
# Add earth engine dataset
dem = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters
dem_vis = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

# Add Earth Engine DEM to map
Map.addLayer(dem, dem_vis, 'SRTM DEM')

In [4]:
path_shapefile = "data/hydromodule_zoning.shp"
ROI = geemap.shp_to_ee(path_shapefile)
Map.addLayer(ROI, {}, 'Region of Interest')

In [5]:
slope = ee.Terrain.slope(dem).clip(ROI)
Map.add_layer(slope, {
    'min': 0, 
    'max': 89.99},
    'Slope')

js_snippet = """
    var slopereclass = ee.Image(1)
      .where(slopesmrt.gt(10).and(slopesmrt.lte(20)), 1)
      .where(slopesmrt.gt(20).and(slopesmrt.lte(30)), 2)
      .where(slopesmrt.gt(30).and(slopesmrt.lte(40)), 3)
      .where(slopesmrt.gt(40).and(slopesmrt.lte(50)), 4)
      .where(slopesmrt.gt(50).and(slopesmrt.lte(90)), 5)
"""
geemap.js_snippet_to_py(
    js_snippet, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True
)

In [6]:
slopereclass = ee.Image(1) \
.where(slope.gt(0).And(slope.lte(1.7)), 1) \
.where(slope.gt(1.7).And(slope.lte(2.9)), 2) \
.where(slope.gt(2.9).And(slope.lte(40)), 3)
# 1 - Қарықпен суару
# 2 - Жаңбырлатып суару
# 3 - Тамшылатып суару

In [7]:
Map.add_layer(slopereclass, {}, 'Classified slope values')

In [8]:
Map

Map(center=[44.79, 77.92], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…